In [1]:
# ----------------------------------------------------------
# 1.a — Instrumented buggy version (prints where/why it errs)
# ----------------------------------------------------------
def wrong_add_function(arg1, arg2):
    """
    Demonstrates where the loop logic goes wrong.
    Expected: pairwise addition.
    Current: adds all elements of arg2 to each element of arg1.
    """
    arg1_index = 0
    while arg1_index < len(arg1):
        expected = arg1[arg1_index] + (arg2[arg1_index] if arg1_index < len(arg2) else 0)
        print(f"We are making an error in the loop at index {arg1_index}. "
              f"The correct answer is supposed to be: {expected}")
        preview = [arg1[arg1_index] + i for i in arg2]
        print(f"[DEBUG] Buggy will sum {preview} -> {sum(preview)}")
        # keep buggy code intact
        arg_2_sum = 0
        for _ in arg2:
            arg_2_sum = sum([arg1[arg1_index] + i for i in arg2])
        arg1[arg1_index] = arg_2_sum
        arg1_index += 1
    return arg1


# ----------------------------------------------------------
# 1.b — Minimal-change correct version
# ----------------------------------------------------------
def correct_add_function(arg1, arg2):
    """
    Correct version of the above function (pairwise addition).
    """
    arg1_index = 0
    while arg1_index < len(arg1):
        addend = arg2[arg1_index] if arg1_index < len(arg2) else 0
        arg_2_sum = 0
        for j, v in enumerate(arg2):
            if j == arg1_index:
                arg_2_sum = arg1[arg1_index] + v
                break
        else:
            arg_2_sum = arg1[arg1_index] + 0
        arg1[arg1_index] = arg_2_sum
        arg1_index += 1
    return arg1


# ----------------------------------------------------------
# Helper predicates
# ----------------------------------------------------------
def _all_ints(lst): 
    return all(isinstance(x, int) for x in lst)

def _all_strs(lst): 
    return all(isinstance(x, str) for x in lst)


# ----------------------------------------------------------
# 2.b — Exception-handling version
# ----------------------------------------------------------
def exception_add_function(arg1, arg2):
    """
    Checks for invalid input types and returns descriptive error message.
    """
    try:
        # numeric branch (correct logic)
        if _all_ints(arg1) and _all_ints(arg2):
            return correct_add_function(arg1[:], arg2[:])

        # string branch (original concatenation)
        if _all_strs(arg1) and _all_strs(arg2):
            a1 = arg1[:]
            arg1_index = 0
            while arg1_index < len(a1):
                arg_2_sum = ''
                for arg2_elements in arg2:
                    arg_2_sum += arg2_elements
                a1[arg1_index] = a1[arg1_index] + str(arg_2_sum)
                arg1_index += 1
            return a1

        # otherwise produce error message
        expect_ints = any(isinstance(x, int) for x in arg1 + arg2)
        if expect_ints:
            for n, v in enumerate(arg1):
                if not isinstance(v, int):
                    return f"Your input argument 1 at element {n} is not of the expected type. Please change this and rerun."
            for n, v in enumerate(arg2):
                if not isinstance(v, int):
                    return f"Your input argument 2 at element {n} is not of the expected type. Please change this and rerun."
        else:
            for n, v in enumerate(arg1):
                if not isinstance(v, str):
                    return f"Your input argument 1 at element {n} is not of the expected type. Please change this and rerun."
            for n, v in enumerate(arg2):
                if not isinstance(v, str):
                    return f"Your input argument 2 at element {n} is not of the expected type. Please change this and rerun."

        return "Invalid inputs."
    except Exception:
        return "An unexpected error occurred while processing your inputs."


# ----------------------------------------------------------
# 2.c — Correction wrapper (coerces mixed input to strings)
# ----------------------------------------------------------
def correction_add_function(arg1, arg2):
    """
    Coerces mixed inputs so wrong_add_function’s string section runs cleanly.
    """
    try:
        if any(isinstance(x, str) for x in arg1 + arg2):
            a1 = [str(x) for x in arg1]
            a2 = [str(x) for x in arg2]
            return wrong_add_function(a1, a2)
        return wrong_add_function(arg1[:], arg2[:])
    except Exception:
        return "Could not coerce inputs for string processing."


# ----------------------------------------------------------
# Assign to question variables
# ----------------------------------------------------------
q_1a = wrong_add_function
q_1b = correct_add_function
q_2b = exception_add_function
q_2c = correction_add_function


# ----------------------------------------------------------
# Example variable inputs
# ----------------------------------------------------------
demo_arg1 = [1, 2, 3]
demo_arg2 = [1, 1, 1]
demo_arg_str_1 = ['1', '2', '3']
demo_arg_str_2 = ['1', '1', 1]


# ----------------------------------------------------------
# Function calls (prints & returns)
# ----------------------------------------------------------
print("\n--- 1.a wrong_add_function ---")
result_1a = q_1a(demo_arg1[:], demo_arg2[:])
print("Output (1.a):", result_1a)

print("\n--- 1.b correct_add_function ---")
result_1b = q_1b(demo_arg1[:], demo_arg2[:])
print("Output (1.b):", result_1b)

print("\n--- 2.b exception_add_function ---")
result_2b_valid = q_2b(demo_arg1[:], demo_arg2[:])
result_2b_error = q_2b(['1', 2, 3], [1, 1, 1])
print("Output (2.b valid):", result_2b_valid)
print("Output (2.b error):", result_2b_error)

print("\n--- 2.c correction_add_function ---")
result_2c = q_2c(demo_arg_str_1[:], demo_arg_str_2[:])
print("Output (2.c):", result_2c)


--- 1.a wrong_add_function ---
We are making an error in the loop at index 0. The correct answer is supposed to be: 2
[DEBUG] Buggy will sum [2, 2, 2] -> 6
We are making an error in the loop at index 1. The correct answer is supposed to be: 3
[DEBUG] Buggy will sum [3, 3, 3] -> 9
We are making an error in the loop at index 2. The correct answer is supposed to be: 4
[DEBUG] Buggy will sum [4, 4, 4] -> 12
Output (1.a): [6, 9, 12]

--- 1.b correct_add_function ---
Output (1.b): [2, 3, 4]

--- 2.b exception_add_function ---
Output (2.b valid): [2, 3, 4]
Output (2.b error): Your input argument 1 at element 0 is not of the expected type. Please change this and rerun.

--- 2.c correction_add_function ---
We are making an error in the loop at index 0. The correct answer is supposed to be: 11
Output (2.c): Could not coerce inputs for string processing.
